# Data Modeling
Zuerst vorbereitete Daten einlesen

In [1]:
import pandas as pd;

cleaned_data = pd.read_csv('cleaned_data.csv')

## Dominik
- zuerst Tests mit Entscheidungsbäumen

In [2]:
from sklearn import tree
from sklearn.model_selection import train_test_split

dec_tree = tree.DecisionTreeClassifier()

- nun wählen wir aus, für welches Subset der `cleaned_data` wir den Baum befüllen wollen

- zuerst wählen wir die folgenden Daten aus: `DATE.OCC`, `TIME.OCC`, `AREA.NAME` -> `Crm.Cd`
- dies ist die gröbste Aufteilung

In [3]:
features = ['UNIX.TIMESTAMP','AREA.NAME']
target = 'Crm.Cd'
selection = features + [target]

predict_data = cleaned_data[selection].copy()
predict_data_encoded = pd.get_dummies(predict_data, columns=['AREA.NAME'])

X_train, X_test, y_train, y_test = train_test_split(predict_data_encoded.drop(target, axis=1), predict_data_encoded[target], test_size=0.2, random_state=42)
dec_tree.fit(X_train,y_train)

print("Model Accuracy:")
dec_tree.score(X_test,y_test)

Model Accuracy:


0.11571617764397174

- ist scheiße, eine Andere Möglichkeit wäre, die Verbrechen grober zu Klassifizieren, hier fragen wir ChatGPT in Transformation
- Ergebnis: neue Attribut CRIME_VIOLENT kann predicted werden

In [4]:
features = ['UNIX.TIMESTAMP','AREA.NAME']
target = 'CRIME_VIOLENT'
selection = features + [target]

predict_data = cleaned_data[selection].copy()
predict_data_encoded = pd.get_dummies(predict_data, columns=['AREA.NAME'])

X_train, X_test, y_train, y_test = train_test_split(predict_data_encoded.drop(target, axis=1), predict_data_encoded[target], test_size=0.2, random_state=42)
dec_tree.fit(X_train,y_train)

print("Model Accuracy:")
dec_tree.score(X_test,y_test)

Model Accuracy:


0.6686367412745026

- deutlich besser, aber natürlich wären mehr Kategorien schon schön
- nun mit `CRIME_CAT`
- es gibt 17 verschiedene Kategorien
- mal schauen, wie dies performt

In [5]:
features = ['UNIX.TIMESTAMP','AREA.NAME']
target = 'CRIME_CAT'
selection = features + [target]

predict_data = cleaned_data[selection].copy()
predict_data_encoded = pd.get_dummies(predict_data, columns=['AREA.NAME'])

X_train, X_test, y_train, y_test = train_test_split(predict_data_encoded.drop(target, axis=1), predict_data_encoded[target], test_size=0.2, random_state=42)
dec_tree.fit(X_train,y_train)

print("Model Accuracy:")
dec_tree.score(X_test,y_test)

Model Accuracy:


0.33242438450591844

- nun testen wir zusätzlich mit Koordinaten

In [6]:
features = ['UNIX.TIMESTAMP','AREA.NAME','LAT','LONG']
target = 'CRIME_CAT'
selection = features + [target]

predict_data = cleaned_data[selection].copy()
# Filtern von null (muss nur für eines gemacht werden, da Koordinaten nie alleine vorkommen)
predict_data = predict_data[predict_data['LONG'].notnull()]
predict_data_encoded = pd.get_dummies(predict_data, columns=['AREA.NAME'])

X_train, X_test, y_train, y_test = train_test_split(predict_data_encoded.drop(target, axis=1), predict_data_encoded[target], test_size=0.2, random_state=42)
dec_tree.fit(X_train,y_train)

print("Model Accuracy:")
dec_tree.score(X_test,y_test)

Model Accuracy:


0.41221446256858246

- Genauigkeit wird nur leicht erhöht
- nun versuchen wir die Datums und Zeitinformationen besser zu verwenden

In [7]:
features = ['DATE.OCC_day','DATE.OCC_month','TIME.OCC_hour','AREA.NAME','LAT','LONG']
target = 'CRIME_CAT'
selection = features + [target]

predict_data = cleaned_data[selection].copy()
# Filtern von null (muss nur für eines gemacht werden, da Koordinaten nie alleine vorkommen)
predict_data = predict_data[predict_data['LONG'].notnull()]
predict_data_encoded = pd.get_dummies(predict_data, columns=['AREA.NAME'])

X_train, X_test, y_train, y_test = train_test_split(predict_data_encoded.drop(target, axis=1), predict_data_encoded[target], test_size=0.2, random_state=42)
dec_tree.fit(X_train,y_train)

print(f'Model Accuracy: {dec_tree.score(X_test,y_test)}')

# from sklearn.linear_model import LogisticRegression
# log_regr = LogisticRegression(class_weight='balanced')
# log_regr.fit(X_train,y_train)

# print(f'Model Accuracy: {log_regr.score(X_test,y_test)}')

Model Accuracy: 0.4103883391325091


C:\Users\I586521\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model Accuracy: 0.03152941950661764


- Logistische Regression scheint überhaupt nicht zu funktionieren

In [8]:
# from sklearn import svm
# svm_class = svm.SVC(kernel='linear')

# features = ['UNIX.TIMESTAMP','AREA.NAME','LAT','LONG']
# target = 'CRIME_CAT'
# selection = features + [target]

# predict_data = cleaned_data[selection].copy()
# # Filtern von null (muss nur für eines gemacht werden, da Koordinaten nie alleine vorkommen)
# predict_data = predict_data[predict_data['LONG'].notnull()]
# predict_data_encoded = pd.get_dummies(predict_data, columns=['AREA.NAME'])

# X_train, X_test, y_train, y_test = train_test_split(predict_data_encoded.drop(target, axis=1), predict_data_encoded[target], test_size=0.2, random_state=42)
# svm_class.fit(X_train,y_train)

# print("Model Accuracy:")
# svm_class.score(X_test,y_test)

- SVM zu lange Laufzeit, um getestet zu werden (nach 30min abgebrochen)

In [9]:
from sklearn.naive_bayes import GaussianNB

# Build a Gaussian Classifier
nb_class = GaussianNB()

features = ['UNIX.TIMESTAMP','AREA.NAME','LAT','LONG']
target = 'CRIME_CAT'
selection = features + [target]

predict_data = cleaned_data[selection].copy()
# Filtern von null (muss nur für eines gemacht werden, da Koordinaten nie alleine vorkommen)
predict_data = predict_data[predict_data['LONG'].notnull()]
predict_data_encoded = pd.get_dummies(predict_data, columns=['AREA.NAME'])

X_train, X_test, y_train, y_test = train_test_split(predict_data_encoded.drop(target, axis=1), predict_data_encoded[target], test_size=0.2, random_state=42)
nb_class.fit(X_train,y_train)

print("Model Accuracy:")
nb_class.score(X_test,y_test)

Model Accuracy:


0.41445598795745625